# LMNA
The LMNA gene encodes lamin A and lamin C. Pathogenic variants in LMNA are associated with at least 11 clinically distinct diseases.

In [1]:
import genophenocorr
from genophenocorr.preprocessing import configure_caching_cohort_creator, load_phenopacket_folder
from genophenocorr.view import CohortViewable
import hpotk
from IPython.display import display, HTML


store = hpotk.configure_ontology_store()
hpo = store.load_minimal_hpo()
print(f'Loaded HPO v{hpo.version}')
print(f"Using genophenocorr version {genophenocorr.__version__}")

/Users/robin/PycharmProjects/genophenocorr/gpc_venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Loaded HPO v2024-04-26
Using genophenocorr version 0.1.1dev


# Settings

In [2]:
LMNA_transcript_id = 'NM_170707.4'
LMNA_protein_id = "NP_733821.1"
fpath_phenopackets = 'input/'

In [4]:
cohort_creator = configure_caching_cohort_creator(hpo)
cohort = load_phenopacket_folder(fpath_phenopackets, cohort_creator)
viewer = CohortViewable(hpo, transcript_id=LMNA_transcript_id)
display(HTML(viewer.process(cohort, LMNA_transcript_id)))

Patients Created: 100%|██████████| 127/127 [00:00<00:00, 667.18it/s]
Validated under none policy
No errors or warnings were found


HPO Term,ID,Annotation Count
Ankle contracture,HP:0034677,39
Elbow contracture,HP:0034391,37
Spinal rigidity,HP:0003306,37
Stiff neck,HP:0025258,36
Proximal muscle weakness in upper limbs,HP:0008997,35
Dilated cardiomyopathy,HP:0001644,27
Atrial fibrillation,HP:0005110,25
Arrhythmia,HP:0011675,23
Lipodystrophy,HP:0009125,21
Proximal muscle weakness in lower limbs,HP:0008994,21


In [5]:
from genophenocorr.view import DiseaseViewable

dis_view = DiseaseViewable(hpo, transcript_id=LMNA_transcript_id)
dis_dict = dis_view._prepare_context(cohort)
display(HTML(dis_view.process(cohort)))

## Configure the analysis

In [6]:
from genophenocorr.analysis import configure_cohort_analysis, CohortAnalysisConfiguration

analysis_config = CohortAnalysisConfiguration()
analysis_config.missing_implies_excluded = True
analysis_config.pval_correction = 'fdr_bh'

analysis = configure_cohort_analysis(cohort, hpo, config=analysis_config)


In [8]:
from genophenocorr.model import VariantEffect
from genophenocorr.analysis.predicate import PatientCategories
from genophenocorr.analysis.predicate.genotype import VariantEffectPredicate

by_missense = analysis.compare_by_variant_effect(VariantEffect.MISSENSE_VARIANT, tx_id=LMNA_transcript_id)
by_missense.summarize(hpo, PatientCategories.YES)

MISSENSE_VARIANT on NM_170707.4                       Yes             No  \
                                                    Count Percent  Count   
Lipodystrophy [HP:0009125]                          10/75     13%  15/35   
Alopecia [HP:0001596]                                 0/3      0%  15/15   
Micrognathia [HP:0000347]                             0/3      0%  15/15   
Proptosis [HP:0000520]                                0/3      0%  15/15   
Failure to thrive [HP:0001508]                        0/3      0%  15/15   
...                                                   ...     ...    ...   
Osteolysis involving bones of the upper limbs [...    0/0      0%  15/15   
Abnormality of the Achilles tendon [HP:0005109]       3/3    100%    2/2   
Skeletal muscle hypertrophy [HP:0003712]              3/3    100%    1/1   
Flexion contracture [HP:0001371]                    35/35    100%    7/7   
Abnormality of the skeletal system [HP:0000924]     37/37    100%  23/23   

MISSENSE_VARIANT on NM_170707.4                                       \
                                                   Percent   p value   
Lipodystrophy [HP:0009125]                             43%  0.001167   
Alopecia [HP:0001596]                                 100%  0.001225   
Micrognathia [HP:0000347]                             100%  0.001225   
Proptosis [HP:0000520]                                100%  0.001225   
Failure to thrive [HP:0001508]                        100%  0.001225   
...                                                    ...       ...   
Osteolysis involving bones of the upper limbs [...    100%  1.000000   
Abnormality of the Achilles tendon [HP:0005109]       100%  1.000000   
Skeletal muscle hypertrophy [HP:0003712]              100%  1.000000   
Flexion contracture [HP:0001371]                      100%  1.000000   
Abnormality of the skeletal system [HP:0000924]       100%  1.000000   

MISSENSE_VARIANT on NM_170707.4                                       
                                                   Corrected p value  
Lipodystrophy [HP:0009125]                                  0.080007  
Alopecia [HP:0001596]                                       0.080007  
Micrognathia [HP:0000347]                                   0.080007  
Proptosis [HP:0000520]                                      0.080007  
Failure to thrive [HP:0001508]                              0.080007  
...                                                              ...  
Osteolysis involving bones of the upper limbs [...          1.000000  
Abnormality of the Achilles tendon [HP:0005109]             1.000000  
Skeletal muscle hypertrophy [HP:0003712]                    1.000000  
Flexion contracture [HP:0001371]                            1.000000  
Abnormality of the skeletal system [HP:0000924]             1.000000  

[457 rows x 6 columns]

In [10]:
by_dis = analysis.compare_disease_vs_genotype(VariantEffectPredicate(transcript_id=LMNA_transcript_id, effect=VariantEffect.MISSENSE_VARIANT))
by_dis.summarize(hpo, PatientCategories.YES)

MISSENSE_VARIANT on NM_170707.4    Yes             No                        \
                                 Count Percent  Count Percent       p value   
OMIM:176670                       0/90      0%  15/37     41%  8.025624e-10   
OMIM:151660                      12/90     13%   0/37      0%  1.807470e-02   
OMIM:181350                      34/90     38%   7/37     19%  5.904112e-02   
OMIM:613205                      13/90     14%   2/37      5%  2.276691e-01   
OMIM:115200                      31/90     34%  13/37     35%  1.000000e+00   

MISSENSE_VARIANT on NM_170707.4                    
                                Corrected p value  
OMIM:176670                          4.012812e-09  
OMIM:151660                          4.518674e-02  
OMIM:181350                          9.840187e-02  
OMIM:613205                          2.845863e-01  
OMIM:115200                          1.000000e+00